In [7]:
import argparse
import numpy as np
from get_data import main
from cnn.training import get_trained_model
import matplotlib.pyplot as plt
import warnings
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin
plt.style.use('ggplot')

In [8]:
!pip install hyperopt

In [9]:
parser = argparse.ArgumentParser()
parser.add_argument('--config', help='Path to the config file', default='./default_config.yml')
parser.add_argument('--log-dir', help='Path to the log directory', default='../log/')
parser.add_argument('--posters', help='Path to the posters', default='../data/posters/')
parser.add_argument('--models-dir', help='Path to the saved models', default='../data/models/')
parser.add_argument('--sets-dir', help='Path to the training and testing sets', default='../data/sets/')
parser.add_argument('--database', help='Path to the databse csv', default='../data/poster_data.csv')
parser.add_argument('--csv', help='Path to the clean csv', default='../data/')
parser.add_argument('-s', '--save', help='Save model', action='store_true')
parser.add_argument('-v', '--verbose', help='Verbose', action='store_true')

str_args = '-v -s'.split()
args, _ = parser.parse_known_args(str_args)

In [10]:
clean_movies, train_posters, train_genres, train_ids, test_posters, test_genres, test_ids, model_name, save_model, verbose = main(args)

  6%|▋         | 773/11935 [00:00<00:01, 7724.30it/s]

Database already cleaned
Posters database downloading


 91%|█████████▏| 10915/11935 [00:01<00:00, 8008.55it/s]

Error HTTP Error 404: Not Found with film 9532


100%|██████████| 11935/11935 [00:01<00:00, 8557.30it/s]


Database downloaded
Training and testing sets already made


In [11]:
resnet, _ = get_trained_model(model_name)

Loading keras ResNet50V2


In [12]:
train_features_resnet = resnet.predict(train_posters).reshape((len(train_posters), -1))
test_features_resnet = resnet.predict(test_posters).reshape((len(test_posters), -1))

KeyboardInterrupt: 

# PCA

In [ ]:
from sklearn.decomposition import PCA

pca_resnet = PCA(n_components=0.98)

In [ ]:
from sklearn.preprocessing import StandardScaler

SC = StandardScaler()
SC.fit(train_features_resnet)
train_features_resnet_stand = SC.transform(train_features_resnet)
test_features_resnet_stand = SC.transform(test_features_resnet)

In [ ]:
pca_resnet.fit(train_features_resnet_stand)

In [ ]:
# train_resnet_pca = pca_resnet.transform(train_features_resnet)
# test_resnet_pca = pca_resnet.transform(test_features_resnet)

train_resnet_pca = pca_resnet.transform(train_features_resnet_stand)
test_resnet_pca = pca_resnet.transform(test_features_resnet_stand)

In [ ]:
print(train_features_resnet.shape[1], train_resnet_pca.shape[1])

# KNN 

In [ ]:
from knn.knn import KNN

k = 7
knn = KNN(k)
knn.fit(train_features_resnet, train_genres)

In [ ]:
from utils.accuracy import mono_label

predicted_genres = knn.predict(test_features_resnet, train_genres)
mono_label(test_genres, predicted_genres)

# XGBoost

Lien pour avoir accès à la courbe de loss : https://machinelearningmastery.com/avoid-overfitting-by-early-stopping-with-xgboost-in-python/

In [ ]:
from xgboost import XGBClassifier, plot_tree

train_genres_flat = np.array([np.nonzero(x)[0] for x in train_genres]).squeeze()
test_genres_flat = np.array([np.nonzero(x)[0] for x in test_genres]).squeeze()

Le bloc suivant calcule le classifier optimal sur la search grid optimal:

pour les hyperparamètres de base :https://towardsdatascience.com/fine-tuning-xgboost-in-python-like-a-boss-b4543ed8b1e

In [1]:
boost = XGBClassifier(objective='multi:softmax', num_classes=7)
boost.fit(train_resnet_pca, train_genres_flat, eval_metric=["merror", "mlogloss"], eval_set=[(train_resnet_pca, train_genres_flat), (test_resnet_pca, test_genres_flat)])

train_genres_flat = np.array([np.nonzero(x)[0] for x in train_genres]).squeeze()
test_genres_flat = np.array([np.nonzero(x)[0] for x in test_genres]).squeeze()


def objective(space):

    warnings.filterwarnings(action='ignore', category=DeprecationWarning)
    classifier = xgb.XGBClassifier(n_estimators = space['n_estimators'],
                            max_depth = int(space['max_depth']),
                            learning_rate = space['learning_rate'],
                            gamma = space['gamma'],
                            min_child_weight = space['min_child_weight'],
                            subsample = space['subsample'],
                            colsample_bytree = space['colsample_bytree'],
                            num_classes=7
                            )
    
    classifier.fit(train_resnet_pca, train_genres_flat)

    # Applying k-Fold Cross Validation
    from sklearn.model_selection import cross_val_score
    accuracies = cross_val_score(estimator = classifier, X = train_resnet_pca, y = train_genres_flat, cv = 10)
    CrossValMean = accuracies.mean()

    print("CrossValMean:", CrossValMean)

    return{'loss':1-CrossValMean, 'status': STATUS_OK }

space = {
    'max_depth' : hp.choice('max_depth', range(3, 30, 1)),
    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.5, 0.01),
    'n_estimators' : hp.choice('n_estimators', range(20, 205, 5)),
    'gamma' : hp.quniform('gamma', 0, 0.50, 0.01),
    'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
    'subsample' : hp.quniform('subsample', 0.1, 1, 0.01),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1.0, 0.01)}

#space = {
#    'max_depth' : hp.choice('max_depth', range(4, 5, 1)),
#    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.03, 0.01),
#    'n_estimators' : hp.choice('n_estimators', range(90, 105, 5)),
#    'gamma' : hp.quniform('gamma', 0.9, 0.92, 0.01),
#    'min_child_weight' : hp.quniform('min_child_weight', 1, 3, 1),
#    'subsample' : hp.quniform('subsample', 0.8, 0.82, 0.01),
#    'colsample_bytree' : hp.quniform('colsample_bytree', 0.9, 0.92, 0.01)}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)

print("Best: ", best)

NameError: name 'XGBClassifier' is not defined

In [ ]:
# Fitting XGBoost to the Training set
boost = XGBClassifier(n_estimators = best['n_estimators'],
                            max_depth = best['max_depth'],
                            learning_rate = best['learning_rate'],
                            gamma = best['gamma'],
                            min_child_weight = best['min_child_weight'],
                            subsample = best['subsample'],
                            colsample_bytree = best['colsample_bytree'],
                            num_classes=7
                     )

boost.fit(train_resnet_pca, train_genres_flat, eval_metric=["merror", "mlogloss"], eval_set=[(train_resnet_pca, train_genres_flat), (test_resnet_pca, test_genres_flat)])

Si on veut calculer un arbre sans l'optimisation des hyperparamètres.

In [ ]:
boost = XGBClassifier(objective='multi:softmax', num_classes=7)
boost.fit(train_resnet_pca, train_genres_flat, eval_metric=["merror", "mlogloss"], eval_set=[(train_resnet_pca, train_genres_flat), (test_resnet_pca, test_genres_flat)])

In [ ]:
predicted_genres = boost.predict(test_resnet_pca)
test_genres_flat = np.array([np.nonzero(x)[0] for x in test_genres]).squeeze()

In [ ]:
# retrieve performance metrics
results = boost.evals_result()
epochs = len(results['validation_0']['merror'])
print(epochs)
x_axis = range(0, epochs)

fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_0']['mlogloss'], label='Train')
ax.plot(x_axis, results['validation_1']['mlogloss'], label='Test')
# plot log loss
plt.ylabel('Log Loss')
plt.title('XGBoost Log Loss')
plt.grid("True")
ax.legend()
plt.show()


# plot classification error
fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_0']['merror'], label='Train')
ax.plot(x_axis, results['validation_1']['merror'], label='Test')
ax.legend()
plt.ylabel('Classification Error')
plt.title('XGBoost Classification Error')
plt.grid("True")
ax.legend()
plt.show()

#### performance

In [ ]:
boost.save_model('current.model')
print("accuracy : ", np.mean(test_genres_flat == predicted_genres))

In [ ]:
predicted_genres_matrix = np.zeros((len(predicted_genres), 7))
for i in range(len(predicted_genres)):
    predicted_genres_matrix[i][predicted_genres[i]] = 1

In [ ]:
ConfusionMatrix_display(test_genres, predicted_genres_matrix, config['genres'], "Performance du random_forest")

#### features d'intérêt

In [ ]:
plt.bar(range(len(boost.feature_importances_)), boost.feature_importances_)
plt.show()
print(np.sum(boost.feature_importances_))

#### Affichage de l'arbre

In [ ]:
plot_tree(boost)
fig = plt.gcf()
fig.set_size_inches(300, 150)

#double click pour la visualisation

# SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier # Regarder les autres multiclass possibles
# J'ai testé vite fait OneVsOne: pb de dimension, autre format d'input attendu

svc = OneVsRestClassifier(SVC(gamma='auto'))
svc.fit(train_resnet_pca, train_genres)

In [ ]:
from utils.accuracy import mono_label

predicted_genres = svc.predict(test_resnet_pca)
mono_label(test_genres, predicted_genres)

# Analyse des résultats

In [ ]:
import yaml

config = yaml.safe_load(open(args.config))
class_names = {config['genres'][k]: k for k in config['genres'].keys()}

In [ ]:
from utils.display.results_display import ConfusionMatrix_display

title = 'ResNet+kNN, k='+str(k) 
ConfusionMatrix_display(test_genres, predicted_genres, config['genres'], title)

In [ ]:
from utils.display.results_display import histogram

histogram(test_genres, predicted_genres, k, config['genres'])

In [ ]:
from utils.display.results_display import plot_neighbors
%matplotlib inline

starting_index = np.random.randint(0, 700)
num_images = 5
plot_neighbors(test_posters, test_genres, class_names, predicted_genres, starting_index, num_images, train_posters, train_genres, knn.neighbors,str(starting_index))

Affichage de la matrice de confusion

## PCA 2D

In [ ]:
import matplotlib.pyplot as plt

def display2D(X_ortho, Y):
    colors = Y.argmax(axis=1)
    
    fig = plt.figure(figsize=(10, 10))
    scatter = plt.scatter(X_ortho[:, 0], X_ortho[:, 1], c=colors)
    plt.legend(*scatter.legend_elements())
    
    plt.show(fig)

In [ ]:
pca = PCA(n_components=2)
# pca.fit(train_posters.reshape(len(train_posters), -1))

train_pca = pca.transform(train_posters.reshape(len(train_posters), -1))
# test_pca = pca.transform(test_posters.reshape(len(train_posters), -1))

display2D(train_pca, train_genres)
display2D(train_resnet_pca[:, :2], train_genres)

#Attention: l'erreur dans la VM vient de la version de matplotlib; et upgrade vers matplotlib 3.1.X pas possible
#Changer version Python...